# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846715843792                   -0.70    4.8
  2   -7.852307721692       -2.25       -1.53    1.0
  3   -7.852646104149       -3.47       -2.53    3.0
  4   -7.852646675840       -6.24       -3.38    2.2
  5   -7.852646686024       -7.99       -4.71    2.2
  6   -7.852646686723       -9.16       -5.27    3.2
  7   -7.852646686729      -11.22       -5.94    1.0
  8   -7.852646686730      -12.42       -7.24    2.5
  9   -7.852646686730      -14.45       -7.52    2.8
 10   -7.852646686730   +  -14.75       -8.71    2.0
 11   -7.852646686730      -14.75       -9.33    1.2
 12   -7.852646686730   +  -14.75      -10.33    2.2
 13   -7.852646686730   +    -Inf      -10.87    2.0
 14   -7.852646686730      -14.75      -11.51    1.5
 15   -7.852646686730      -14.75      -12.63    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846838225685                   -0.70           4.5
  2   -7.852527373029       -2.24       -1.63   0.80    2.0
  3   -7.852637216335       -3.96       -2.70   0.80    1.0
  4   -7.852646478874       -5.03       -3.31   0.80    2.2
  5   -7.852646682659       -6.69       -4.15   0.80    1.5
  6   -7.852646686396       -8.43       -4.84   0.80    1.5
  7   -7.852646686725       -9.48       -5.79   0.80    1.8
  8   -7.852646686730      -11.33       -6.67   0.80    1.8
  9   -7.852646686730      -12.79       -7.53   0.80    2.2
 10   -7.852646686730   +  -14.75       -8.19   0.80    1.8
 11   -7.852646686730      -14.45       -8.64   0.80    1.8
 12   -7.852646686730   +  -14.35       -9.06   0.80    2.0
 13   -7.852646686730      -15.05       -9.70   0.80    1.2
 14   -7.852646686730   +  -14.45      -10.62   0.80    1.5
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.362769e+01     3.352338e+00
 * time: 0.42677807807922363
     1     1.369133e+00     1.814754e+00
 * time: 0.6426291465759277
     2    -1.681325e+00     1.890511e+00
 * time: 0.6655511856079102
     3    -3.680314e+00     1.642429e+00
 * time: 0.6983330249786377
     4    -5.164403e+00     1.405626e+00
 * time: 0.7308170795440674
     5    -6.751470e+00     1.031307e+00
 * time: 0.7635650634765625
     6    -7.383428e+00     4.006141e-01
 * time: 0.7960669994354248
     7    -7.606673e+00     1.274471e-01
 * time: 0.8187470436096191
     8    -7.706538e+00     1.635058e-01
 * time: 0.8414649963378906
     9    -7.769154e+00     9.435211e-02
 * time: 0.8641071319580078
    10    -7.813657e+00     7.185459e-02
 * time: 0.8867261409759521
    11    -7.834894e+00     4.148534e-02
 * time: 0.9094321727752686
    12    -7.845373e+00     3.397199e-02
 * time: 0.9322700500488281
    13    -7.849708e+00     3.236356e-02
 * time: 0.954857110

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846797779814                   -0.70    4.5
  2   -7.852299466210       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686495                   -2.55
  2   -7.852646686730       -9.63       -5.79
  3   -7.852646686730   +  -14.57      -11.09
  4   -7.852646686730      -14.75      -15.71


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.132626330642102e-13
|ρ_newton - ρ_scfv| = 4.4157343716498814e-13
|ρ_newton - ρ_dm|   = 1.0715459499968605e-9
